In [1]:
import wandb
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import wandb
import pandas as pd
import matplotlib.pyplot as plt

In [3]:

# Initialize wandb API
api = wandb.Api()

# Replace 'your_sweep_id' with your actual sweep ID
sweep_id = 'maximes_crew/S3_SC_runs/ubr4e2bd'

# Fetch the sweep object
sweep = api.sweep(sweep_id)

# Fetch all runs in the sweep
runs = sweep.runs

sweep_params = ['lr', 'pdrop', 'dt_train', 'inp_b']

In [4]:
runs

<Runs maximes_crew/S3_SC_runs>

In [5]:
# Initialize an empty DataFrame to hold the relevant data
data = []

# Extract the necessary data from each run
for run in runs:
    config = run.config
    summary = run.summary
    history = run.history()
    n_layers = config.get('nb_layers')
    n_hiddens = config.get('nb_hiddens')
    lr = config.get('lr')
    dt_min = config.get('dt_min')
    dt_max = config.get('dt_max')
    pdrop = config.get('pdrop')
    dt_train = config.get('dt_train')
    inp_b = config.get('inp_b')
    sched_patience = config.get('scheduler_patience')
    sched_factor = config.get('scheduler_factor')
    if 'valid acc' in history.columns and 'test acc' in history.columns:
        best_valid_acc_idx = history['valid acc'].idxmax()  # Get the index of the best valid accuracy
        if (best_valid_acc_idx+1) in history['test acc'].index:
            test_acc = history['test acc'].loc[best_valid_acc_idx+1]  # Get test acc at that index
        else:
            test_acc = None
    else:
        test_acc = None

    if test_acc is not None:
        data.append({
            'n_layers': n_layers,
            'n_hiddens': n_hiddens,
            'lr': lr,
            'dt_min': dt_min,
            'dt_max': dt_max,
            'pdrop' : pdrop,
            'dt_train':dt_train,
            'inp_b':inp_b,
            'scheduler_patience' : sched_patience,
            'scheduler_factor' : sched_factor,
            'test_acc': test_acc
        })



In [6]:
data

[{'n_layers': 3,
  'n_hiddens': 512,
  'lr': 0.0005,
  'dt_min': 0.01,
  'dt_max': 0.5,
  'pdrop': 0.5,
  'dt_train': False,
  'inp_b': False,
  'scheduler_patience': 5,
  'scheduler_factor': 0.7,
  'test_acc': 0.9519252455894146},
 {'n_layers': 3,
  'n_hiddens': 512,
  'lr': 0.0005,
  'dt_min': 0.01,
  'dt_max': 0.5,
  'pdrop': 0.5,
  'dt_train': False,
  'inp_b': False,
  'scheduler_patience': 5,
  'scheduler_factor': 0.7,
  'test_acc': 0.9485640537289496},
 {'n_layers': 3,
  'n_hiddens': 512,
  'lr': 0.0005,
  'dt_min': 0.01,
  'dt_max': 0.5,
  'pdrop': 0.5,
  'dt_train': False,
  'inp_b': False,
  'scheduler_patience': 5,
  'scheduler_factor': 0.7,
  'test_acc': 0.951189103849238},
 {'n_layers': 3,
  'n_hiddens': 512,
  'lr': 0.0005,
  'dt_min': 0.01,
  'dt_max': 0.5,
  'pdrop': 0.5,
  'dt_train': False,
  'inp_b': False,
  'scheduler_patience': 5,
  'scheduler_factor': 0.7,
  'test_acc': 0.9529245188452286},
 {'n_layers': 3,
  'n_hiddens': 512,
  'lr': 0.0005,
  'dt_min': 0.01,
  

In [9]:
import itertools

# Define the grid sweep parameters
param_grid = {
    "lr": [0.01, 0.005, 0.001],
    "n_layers": [2, 3, 4, 5, 6, 7],
    "n_layers": [2, 3, 4, 5, 6, 7],
    "pdrop": [0.1, 0.25, 0.5],
}

# List of all launched runs (as a list of dictionaries containing the parameters used in each run)
launched_runs = data

# Generate all possible parameter combinations (the full grid)
all_combinations = list(itertools.product(
    param_grid['lr'], param_grid['n_layers'], param_grid['pdrop']
))

# Function to convert tuples to dictionaries for easy comparison
def tuple_to_dict(param_tuple):
    return {
        "lr": param_tuple[0],
        "n_layers": param_tuple[1],
        "pdrop": param_tuple[2],
    }

# Convert all combinations to dicts
all_combinations_as_dicts = [tuple_to_dict(combo) for combo in all_combinations]

# Extract relevant fields from launched runs
def extract_relevant_params(launched_run):
    return {
        "lr": launched_run['lr'],
        "n_layers": launched_run['n_layers'],
        "pdrop": launched_run['pdrop'],
    }

# Extract relevant parameters from launched runs for comparison
launched_params = [extract_relevant_params(run) for run in launched_runs]

# Find missing runs
missing_runs = [combo for combo in all_combinations_as_dicts if combo not in launched_params]


In [10]:
len(launched_params)

109

In [11]:
len(all_combinations_as_dicts)

54

In [18]:
len(missing_runs)

50

In [11]:
# Convert the data to a DataFrame for easier manipulation
df_all = pd.DataFrame(data)

In [12]:
# Group by the sweep params and model_name, then compute the mean of test_acc for each group
df_grouped = df_all.groupby(sweep_params).agg({'test_acc': ['mean', 'std']}).reset_index()

# Calculate the number of runs for each unique configuration
df_all['run_count'] = df_all.groupby(sweep_params).transform('size')

df_grouped['run_count'] = df_grouped.apply(lambda x: df_all[
    (df_all[sweep_params].apply(tuple, axis=1) == tuple(x[sweep_params]))
]['run_count'].iloc[0], axis=1)

# Sort the grouped DataFrame by test_acc in descending order
df_grouped_sorted = df_grouped.sort_values(by=('test_acc', 'mean'), ascending=False).reset_index(drop=True)

In [13]:
df_grouped_sorted[0:10]

lr pdrop dt_train  inp_b  test_acc           run_count
                                     mean       std          
0  0.0010  0.50    False  False  0.952730  0.000550         5
1  0.0010  0.50     True  False  0.952400  0.000731         5
2  0.0050  0.50    False  False  0.951802  0.001778         5
3  0.0005  0.50     True  False  0.951362  0.000744         5
4  0.0050  0.50     True  False  0.951071  0.001105         5
5  0.0005  0.50    False  False  0.951033  0.001637         5
6  0.0010  0.50    False   True  0.951015  0.000768         5
7  0.0005  0.25     True  False  0.950839  0.000871         5
8  0.0005  0.50    False   True  0.950603  0.001850         5
9  0.0010  0.50     True   True  0.950399  0.000898         5